In [ ]:


!pip install pyarrow

import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define file paths
# NOTE: change month for each year(7,8,9,10)
file1 = '/content/drive/My Drive/GEE_Exports/Alberta/2024/landcover_filtered_merged-05.parquet'  # Update this path
file2 = '/content/drive/MyDrive/GEE_Exports/Alberta/2024/ERA5/weatherData-2024-05.parquet'  # Update path
output_file = '/content/drive/My Drive/GEE_Exports/Alberta/2024/final_final_file-2024-05.parquet'  # Update path

BATCH_SIZE = 80000  # Adjust based on memory capacity

Mounted at /content/drive


In [ ]:
# Load the smaller dataset (file2) fully into memory
print("Reading the second file into memory...")
df2 = pd.read_parquet(file2)
df2['date'] = pd.to_datetime(df2['date'])


Reading the second file into memory...


In [ ]:

# Open file1 as a dataset to process in chunks
print("Processing the first file in chunks...")
dataset1 = ds.dataset(file1, format="parquet")

# Initialize ParquetWriter for incremental writes
writer = None




Processing the first file in chunks...


In [ ]:
# Process in chunks
for batch in dataset1.to_batches(batch_size=BATCH_SIZE):  # Adjust batch size based on memory capacity
    # Convert batch to a Pandas DataFrame
    df1 = batch.to_pandas()

    # Ensure the 'date' column is in datetime format
    df1['date'] = pd.to_datetime(df1['date'])

    # Perform the merge (left join to add new features from df2)
    merged_df = pd.merge(df1, df2, on=["date", "latitude", "longitude"], how="left")

    # Convert the merged DataFrame back to an Arrow Table
    merged_table = pa.Table.from_pandas(merged_df)

    # Write to the output Parquet file incrementally
    if writer is None:
        writer = pq.ParquetWriter(output_file, merged_table.schema)
    writer.write_table(merged_table)

# Close the writer
if writer is not None:
    writer.close()

print(f"Merged dataset saved to {output_file}")

Merged dataset saved to /content/drive/My Drive/GEE_Exports/Alberta/2024/final_final_file-2024-05.parquet
